In [152]:
# Load data
import re
from itertools import chain # this consumes iterable

with open("../data/day16.txt", "r") as f:
    input = f.read()

input_list = input.split("\n\n")
rules = input_list[0].split("\n")
#my_ticket = input_list[1].split("\n")[1].split(",")
nearby_tickets = [[int(value) for value in ticket.split(",")] for ticket in input_list[2].split("\n")[1:]]

regex = re.compile(r"(.*): (\d+)-(\d+) or (\d+)-(\d+)") # named groups to dict?
rule_dict = {}
for rule in rules:
    data = regex.search(rule).groups()
    rule_name = data[0]
    allowed = list(range(int(data[1]), int(data[2]) + 1)) + list(range(int(data[3]), int(data[4]) + 1))
    rule_dict[rule_name] = allowed

full_rule_set = set().union(*rule_dict.values())

#zone: 34-521 or 534-971
assert 33 not in rule_dict["zone"] 
assert 34 in rule_dict["zone"]
assert 521 in rule_dict["zone"]
assert 522 not in rule_dict["zone"]
assert 533 not in rule_dict["zone"]
assert 534 in rule_dict["zone"]
assert 971 in rule_dict["zone"]
assert 972 not in rule_dict["zone"]

In [163]:
# Puzzle 1
errors = []
valid_tickets = []
for ticket in nearby_tickets:
    error = False
    for value in ticket:
        error = value not in full_rule_set
        if error:
            errors.append(value)
    if error:
        valid_tickets.append(ticket)

print(sum(errors))

21081


In [ ]:
# Puzzle 2

# Discard invalid tickets

In [161]:
nearby_tickets


  68,
  299,
  731,
  701,
  625,
  113,
  503,
  617,
  747,
  497],
 [53,
  376,
  540,
  383,
  336,
  780,
  136,
  925,
  353,
  805,
  346,
  747,
  411,
  911,
  386,
  542,
  116,
  913,
  772,
  803],
 [382,
  508,
  553,
  893,
  490,
  377,
  186,
  924,
  121,
  573,
  379,
  420,
  306,
  1,
  739,
  589,
  128,
  379,
  734,
  434],
 [903,
  383,
  741,
  350,
  103,
  350,
  179,
  407,
  72,
  521,
  628,
  773,
  646,
  401,
  755,
  236,
  61,
  372,
  812,
  330],
 [740,
  565,
  373,
  400,
  491,
  563,
  737,
  625,
  461,
  761,
  67,
  303,
  913,
  979,
  383,
  844,
  711,
  357,
  865,
  740],
 [701,
  243,
  62,
  760,
  126,
  684,
  343,
  149,
  982,
  444,
  179,
  693,
  212,
  902,
  421,
  347,
  100,
  341,
  617,
  498],
 [270,
  796,
  626,
  699,
  518,
  735,
  205,
  559,
  13,
  842,
  896,
  909,
  237,
  809,
  102,
  376,
  809,
  558,
  471,
  462],
 [346,
  565,
  630,
  97,
  382,
  393,
  270,
  904,
  300,
  199,
  465,
  941,
  497,
 